# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [1]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler,LabelEncoder,MinMaxScaler
from scipy import stats
from sklearn.cluster import KMeans
from scipy.stats import zscore
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer

### 2. Load the dataset

In [2]:
df= pd.read_csv('car_data.csv')

In [3]:
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [4]:
df.shape

(301, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [6]:
len(df[df.duplicated()])

2

In [7]:
df.drop_duplicates(keep='first',inplace=True)


In [8]:
len(df[df.duplicated()])

0

### 5. Drop the columns which you think redundant for the analysis.

In [9]:
df.columns

Index(['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner'],
      dtype='object')

In [10]:
df['Seller_Type'].unique()

array(['Dealer', 'Individual'], dtype=object)

In [11]:
df['Owner'].value_counts()

0    288
1     10
3      1
Name: Owner, dtype: int64

In [12]:
df.drop(['Car_Name'],axis=1,inplace=True)

In [13]:
df.columns

Index(['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven', 'Fuel_Type',
       'Seller_Type', 'Transmission', 'Owner'],
      dtype='object')

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [14]:
# df['age_of_the_car']=2022-df['Year']

In [15]:
# df['age_of_the_car']

In [16]:
from datetime import date
 

today = date.today().year
df['Age_of_the_Car']=today-df['Year']
df['Age_of_the_Car']


0       8
1       9
2       5
3      11
4       8
       ..
296     6
297     7
298    13
299     5
300     6
Name: Age_of_the_Car, Length: 299, dtype: int64

In [17]:
df.drop(['Year'],axis=1,inplace=True)
df

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Age_of_the_Car
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,8
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,9
2,7.25,9.85,6900,Petrol,Dealer,Manual,0,5
3,2.85,4.15,5200,Petrol,Dealer,Manual,0,11
4,4.60,6.87,42450,Diesel,Dealer,Manual,0,8
...,...,...,...,...,...,...,...,...
296,9.50,11.60,33988,Diesel,Dealer,Manual,0,6
297,4.00,5.90,60000,Petrol,Dealer,Manual,0,7
298,3.35,11.00,87934,Petrol,Dealer,Manual,0,13
299,11.50,12.50,9000,Diesel,Dealer,Manual,0,5


### 7. Encode the categorical columns

In [18]:
df.select_dtypes('object').columns

Index(['Fuel_Type', 'Seller_Type', 'Transmission'], dtype='object')

In [19]:
for col in df.select_dtypes('object').columns:
    le=LabelEncoder()
    df[col]=le.fit_transform(df[col])

In [20]:
df

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Age_of_the_Car
0,3.35,5.59,27000,2,0,1,0,8
1,4.75,9.54,43000,1,0,1,0,9
2,7.25,9.85,6900,2,0,1,0,5
3,2.85,4.15,5200,2,0,1,0,11
4,4.60,6.87,42450,1,0,1,0,8
...,...,...,...,...,...,...,...,...
296,9.50,11.60,33988,1,0,1,0,6
297,4.00,5.90,60000,2,0,1,0,7
298,3.35,11.00,87934,2,0,1,0,13
299,11.50,12.50,9000,1,0,1,0,5


### 8. Separate the target and independent features.

In [21]:
X = df.drop('Selling_Price',axis=1)
y = df['Selling_Price']

In [22]:
X

,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Age_of_the_Car
0,5.59,27000,2,0,1,0,8
1,9.54,43000,1,0,1,0,9
2,9.85,6900,2,0,1,0,5
3,4.15,5200,2,0,1,0,11
4,6.87,42450,1,0,1,0,8
...,...,...,...,...,...,...,...
296,11.60,33988,1,0,1,0,6
297,5.90,60000,2,0,1,0,7
298,11.00,87934,2,0,1,0,13
299,12.50,9000,1,0,1,0,5


In [23]:
y.head()

0    3.35
1    4.75
2    7.25
3    2.85
4    4.60
Name: Selling_Price, dtype: float64

### 9. Split the data into train and test.

In [24]:

 X_train, X_test,y_train, y_test =train_test_split(X,y,test_size=0.30,random_state=1)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(209, 7) (90, 7)
(209,) (90,)


In [25]:
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# print(X_train.shape,X_test.shape)
# print(y_train.shape,y_test.shape)

### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [26]:
from sklearn.metrics import r2_score
def fit_n_predict(model,X_train,X_test,y_train,y_test):
    
    # Fit the model with train data
    model.fit(X_train,y_train)
    
    # Making prediction on test data
    pred=model.predict(X_test)
    
    # Calculate the accuracy score
    accuracy=r2_score(y_test,pred)
    
    return accuracy

In [27]:
# Testing the above function
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

rs = pd.DataFrame()

result= fit_n_predict(rf,X_train,X_test,y_train,y_test)

result

0.911787383535449

In [28]:
rs['random_forest']= pd.Series(result)

In [29]:
rs

,random_forest
0,0.911787


### 11. Create a pickle file with an extension as .pkl

In [30]:
import pickle
# Saving model to disk
pickle.dump(rf, open('model.pkl','wb'))

# # Loading model to compare the results
# model = pickle.load(open('model.pkl','rb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [3]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
   
    <div class="hero-image">
        <div class="hero-text ">
            <h1 style = "font-size:50px"> Used Car Price Predictor </h1>
            <br><br>
            <h3>{{prediction_text}}</h3>
        </div>
    </div>
    <style>
           
           body.html{
            height: 100%;
            margin: 0;
            font-family: Arial, Helvetica, sans-serif;
           }

        .hero-image{
            height: 30%;
            background-position:bottom;
            background-repeat: no-repeat;
            background-size: cover;
            background-color: gray;
            position: relative;
            padding: 10%;
        }
        .hero-text{
            text-align:center;
            position: absolute;
            background-color:gray;
            top: 60%;
            left: 50%;
            transform: translate(-60%,-60%);
            color: white;
            padding: 2%;
        }
    </style>
   
    <div style="color: rgb(0,0,0)">
        <form action="{{url_for('predict')}}", method="post">
            <h2>Enter car Details:</h2>
            <h3>Age of the car(In years)</h3>
            <input id="first" name="Age_of_the_Car" type="number">
            <h3>Present Showroom Price(In lakhs)</h3>
            <input id="second" name="Present_Price" required="required">
            <h3>Kilometers driven</h3>
            <input id="third" name="Kms_Driven" required="required">
            <h3>Owner Type(0/1/3)</h3><br>
            <input id="fourth" name="Owner" required="required">
            <h3>Fuel Type</h3>
            <select id="fuel" name="Fuel_Type" required="required">
                <option value="0">Petrol</option>
                <option value ="1">Diesel</option>
                <option value="2">CNG</option>
            </select>  
            <h3>Seller Type</h3>
            <select name="Seller_Type" id="resea" required="required">
                <option value="0">Dealer</option>
                <option value="1">Individual</option>
            </select>
        </select>  
        <h3>Transmission Type</h3>
        <select name="Transmission" id="research" required="required">
            <option value="0">Manual</option>
            <option value="1">automatic car</option>
        </select>
        <br><br>
        <button id ="sub" type="submit">Predict Seller Price</button>
            
        </form>

    </div>

    <style>
    
        body{
            background-color: 101,10,20;
            text-align: center;
            padding: 10px; 
            font-family: Helvetica ;

        }
        #research{
            font-size: 18px;
            width: 200px;
            height: 23px;
            top: 23px;

        }
        #box{
            border-radius: 60px;
            border-color: 45px;
            border-style: solid;
            text-align: center;
            background-color: white;
            font-size: medium;
            position: absolute;
            width: 700px;
            bottom: 9%;
            height: 850px;
            right: 30%;
            padding: 0px;
            margin: 0px;
            font-size: 14px;
        }
        #fuel{
            width: 83px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 20px;
        }
        #fuel:hover{
            background-color: white;
        }
        #research{
            width: 150px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }
        #research:hover{
            background-color: white;
        }
        #resea{
            width: 99px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }
        #resea:hover{
            background-color: white;
        }
        #sub{
            background-color: green;
            font-family: 'Helvetica' monospace;
            font-weight: bold;
            width: 180px;
            height: 60px;
            text-align: center;
            border-radius: 20px;
            font-size: 18px;
            color: white;
        }
        #sub:hover{
            background-color: black;

        }
        #first{
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
        #second{
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
        #third{
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
        #fourth{
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
    </style>
</body>
</html>



### b) Create app.py file and write the predict function

In [5]:
# importing necessary libraries and functions
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import sklearn

app = Flask(__name__) #Initialize the flask App
model = pickle.load(open('model.pkl', 'rb')) # loading the trained model

@app.route('/',methods=['GET']) # Homepage
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    if request.method == 'POST':
        Present_Price=float(request.form['Present_Price'])
        Kms_Driven = int(request.form['Kms_Driven'])
        Owner = int(request.form['Owner'])
        Fuel_Type = request.form['Fuel_Type']
        Age_of_the_Car = request.form['Age_of_the_Car']
        Seller_Type=request.form['Seller_Type']
        Transmission=request.form['Transmission']
    
    
    prediction = model.predict([[Present_Price,Kms_Driven,Owner,Fuel_Type,Age_of_the_Car,Seller_Type,Transmission]]) # making prediction

    output=round(prediction[0],2)
    return render_template('index.html', prediction_text="You can sell your car at {} lakhs".format(output)) # rendering the predicted result

if __name__ == "__main__":
    app.run(debug=True)

### 13. Deploy your app on Heroku. (write commands for deployment)

In [ ]:
#create procfile
echo > Procfile(#creates a procfile)
web: gunicorn app:app(# paste this in the procfile)

In [ ]:
#create a runtime.txt file(need to tell what typing of python need to be used)
python-version(pass our version of python in the file)

In [ ]:
#creates requirements.txt file
pip freeze > requirements.txt

In [ ]:
#initialize a git repository
 git init

In [ ]:
#push your code to git repository
git add .
git commit -m 'Initial commit'

In [ ]:
# Push the code from local repository's main branch to your heroku remote
heroku git:remote -a nameofyourapp(this will set git remotebheroku)
git push heroku master

### 14. Paste the URL of the heroku application below, and while submitting the solution submit this notebook along with the source code.

In [ ]:
https://predict-carprice-2023.herokuapp.com/

### Happy Learning :)